In [1]:
import pandas as pd  # Mengimpor pandas untuk manipulasi data seperti membaca dan mengolah dataset
import torch  # Mengimpor PyTorch untuk komputasi tensor dan deep learning
import torch.nn as nn  # Mengimpor modul neural network (nn) dari PyTorch untuk membangun model neural network
import torch.optim as optim  # Mengimpor optimizers dari PyTorch untuk mengoptimalkan model
from torch.utils.data import DataLoader, Dataset  # Mengimpor DataLoader dan Dataset untuk manajemen data dalam PyTorch
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Mengimpor preprocessing tools dari scikit-learn (LabelEncoder untuk encoding label, StandardScaler untuk normalisasi data)
from sklearn.model_selection import train_test_split  # Mengimpor fungsi untuk membagi dataset menjadi data latih dan data uji
import numpy as np  # Mengimpor NumPy untuk manipulasi array dan operasi matematis
from sklearn.metrics import accuracy_score  # Mengimpor fungsi untuk menghitung akurasi model

In [3]:
# Load dataset
file_path = '/content/ai4i2020.csv'
data1 = pd.read_csv(file_path)

In [5]:
# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Identifikasi kolom kategorikal dan lakukan encoding
kolom_kategorikal = ['UDI', 'Product ID', 'Type']
for kolom in kolom_kategorikal:
    data1[kolom] = label_encoder.fit_transform(data1[kolom])

# Pisahkan fitur dan target
X = data1.drop(columns=['Machine failure'])  # Menghapus kolom target dari fitur
y = data1['Machine failure']

# Isi nilai yang hilang dan normalisasi fitur
X = X.fillna(0)  # Ganti nilai yang hilang dengan 0
scaler = StandardScaler()

# Hanya skala kolom numerik
# Dapatkan daftar kolom numerik
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Konversi data ke tensor PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32) # Use .values to get NumPy array
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)  # Use .values to get NumPy array
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Pindahkan tensor ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)


Kode ini memproses dataset dengan langkah-langkah sistematis, dimulai dengan encoding kolom kategorikal seperti UDI, Product ID, dan Type menggunakan LabelEncoder. Kolom target Machine failure dipisahkan dari fitur, dan nilai yang hilang pada dataset diisi dengan nol. Fitur numerik dinormalisasi menggunakan StandardScaler untuk memastikan skala yang konsisten. Dataset kemudian dibagi menjadi data latih dan uji dengan rasio 80:20 menggunakan train_test_split. Data latih dan uji dikonversi menjadi tensor PyTorch, yang kemudian dipindahkan ke GPU jika tersedia, untuk mempercepat proses pelatihan dan evaluasi model.

In [6]:
# Model RNN dengan pooling
class ModelMarkovRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling):
        super(ModelMarkovRNN, self).__init__()
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, batch_first=True)  # RNN sederhana
        self.jenis_pooling = jenis_pooling  # Jenis pooling: 'max' atau 'avg'
        self.fc = nn.Linear(ukuran_hidden, jumlah_kelas)  # Lapisan fully connected untuk klasifikasi

    def forward(self, x):
        out, _ = self.rnn(x)  # Forward pass melalui RNN
        if self.jenis_pooling == 'max':
            out = torch.max(out, dim=1).values  # Pooling maksimum
        elif self.jenis_pooling == 'avg':
            out = torch.mean(out, dim=1)  # Pooling rata-rata
        out = self.fc(out)  # Lapisan klasifikasi akhir
        return out

In [7]:
# Hyperparameter eksperimen
ukuran_input = X_train_tensor.shape[1]  # Jumlah fitur input
jumlah_kelas = len(label_encoder.classes_)  # Jumlah kelas output
ukuran_hidden_list = [32, 64, 128]  # Daftar ukuran hidden layer yang akan diuji
jenis_pooling_list = ['max', 'avg']  # Metode pooling yang akan diuji
daftar_epoch = [5, 50, 100, 250, 350]  # Daftar jumlah epoch yang akan diuji
daftar_optimizer = ['SGD', 'RMSprop', 'Adam']  # Daftar optimizer yang akan diuji

In [8]:
# Variabel untuk menyimpan model terbaik
model_terbaik = None
akurasi_terbaik = 0
parameter_terbaik = {}
ringkasan_hasil = []  # Untuk menyimpan ringkasan hasil

Kode ini memproses dataset dengan langkah-langkah sistematis, dimulai dengan encoding kolom kategorikal seperti UDI, Product ID, dan Type menggunakan LabelEncoder. Dataset kemudian dipisahkan menjadi fitur (X) dan target (y), di mana kolom target adalah Machine failure. Nilai yang hilang pada dataset diisi menggunakan angka nol, dan fitur numerik dinormalisasi menggunakan StandardScaler. Setelah itu, dataset dibagi menjadi data latih dan uji dengan rasio 80:20 menggunakan train_test_split. Data latih dan uji kemudian dikonversi menjadi tensor PyTorch untuk kompatibilitas dengan model pembelajaran mendalam, dan tensor dipindahkan ke GPU jika tersedia untuk mempercepat pelatihan. Proses ini memastikan dataset siap digunakan untuk pelatihan dan evaluasi model.

In [9]:
# Eksperimen kombinasi hyperparameter
for ukuran_hidden in ukuran_hidden_list:
    for jenis_pooling in jenis_pooling_list:
        for nama_optimizer in daftar_optimizer:
            for epoch in daftar_epoch:
                print(f"--- Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch} ---")

                # Inisialisasi model dan fungsi loss
                model = ModelMarkovRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)
                fungsi_loss = nn.CrossEntropyLoss()

                # Pilih optimizer
                if nama_optimizer == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=0.01)
                elif nama_optimizer == 'RMSprop':
                    optimizer = optim.RMSprop(model.parameters(), lr=0.01)
                elif nama_optimizer == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.01)

                # Pengaturan Early Stopping
                batas_sabar = 10
                loss_terbaik = float('inf')
                penghitung_sabar = 0

                # Loop pelatihan
                for ep in range(epoch):
                    model.train()  # Mode pelatihan
                    optimizer.zero_grad()  # Bersihkan gradien
                    output = model(X_train_tensor.unsqueeze(1))  # Forward pass
                    loss = fungsi_loss(output, y_train_tensor)  # Hitung loss
                    loss.backward()  # Backpropagation
                    optimizer.step()  # Perbarui parameter

                    # Validasi
                    model.eval()  # Mode evaluasi
                    with torch.no_grad():
                        val_output = model(X_test_tensor.unsqueeze(1))  # Forward pass validasi
                        val_loss = fungsi_loss(val_output, y_test_tensor).item()  # Hitung loss validasi
                        prediksi_val = val_output.argmax(dim=1)  # Prediksi kelas
                        akurasi_val = accuracy_score(y_test_tensor.cpu(), prediksi_val.cpu()) * 100  # Akurasi

                    # Log detail epoch
                    print(f"Epoch {ep + 1}/{epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Akurasi: {akurasi_val:.2f}%")

                    # Cek Early Stopping
                    if val_loss < loss_terbaik:
                        loss_terbaik = val_loss
                        penghitung_sabar = 0
                    else:
                        penghitung_sabar += 1

                    if penghitung_sabar >= batas_sabar:
                        print("Early stopping diaktifkan.")
                        break

                # Evaluasi model akhir
                model.eval()
                with torch.no_grad():
                    prediksi = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)  # Prediksi pada test set
                    akurasi = accuracy_score(y_test_tensor.cpu(), prediksi.cpu())  # Hitung akurasi

                # Log akurasi akhir untuk kombinasi hyperparameter saat ini
                print(f"Akurasi Akhir untuk Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch}: {akurasi:.6f}")
                ringkasan_hasil.append(f"{ukuran_hidden},{jenis_pooling},{nama_optimizer},{epoch},{akurasi}")

                # Simpan model dan parameter terbaik
                if akurasi > akurasi_terbaik:
                    akurasi_terbaik = akurasi
                    model_terbaik = model
                    parameter_terbaik = {
                        'ukuran_hidden': ukuran_hidden,
                        'jenis_pooling': jenis_pooling,
                        'optimizer': nama_optimizer,
                        'epoch': epoch
                    }

Streaming output truncated to the last 5000 lines.
Epoch 5/100, Loss: 0.3777, Val Loss: 0.2749, Akurasi: 99.60%
Epoch 6/100, Loss: 0.2743, Val Loss: 0.1925, Akurasi: 99.60%
Epoch 7/100, Loss: 0.1913, Val Loss: 0.1309, Akurasi: 99.60%
Epoch 8/100, Loss: 0.1293, Val Loss: 0.0876, Akurasi: 99.60%
Epoch 9/100, Loss: 0.0860, Val Loss: 0.0590, Akurasi: 99.60%
Epoch 10/100, Loss: 0.0573, Val Loss: 0.0408, Akurasi: 99.60%
Epoch 11/100, Loss: 0.0391, Val Loss: 0.0293, Akurasi: 99.90%
Epoch 12/100, Loss: 0.0278, Val Loss: 0.0222, Akurasi: 99.90%
Epoch 13/100, Loss: 0.0208, Val Loss: 0.0178, Akurasi: 99.90%
Epoch 14/100, Loss: 0.0164, Val Loss: 0.0150, Akurasi: 99.90%
Epoch 15/100, Loss: 0.0136, Val Loss: 0.0132, Akurasi: 99.90%
Epoch 16/100, Loss: 0.0118, Val Loss: 0.0121, Akurasi: 99.90%
Epoch 17/100, Loss: 0.0107, Val Loss: 0.0114, Akurasi: 99.90%
Epoch 18/100, Loss: 0.0100, Val Loss: 0.0110, Akurasi: 99.90%
Epoch 19/100, Loss: 0.0095, Val Loss: 0.0107, Akurasi: 99.90%
Epoch 20/100, Loss: 0.00

Kode ini mendefinisikan model Recurrent Neural Network (RNN) sederhana melalui kelas ModelMarkovRNN, yang memiliki lapisan RNN untuk memproses data sekuensial dan lapisan fully connected untuk klasifikasi. Model ini mendukung penggunaan pooling maksimum (max) dan rata-rata (avg) untuk merangkum keluaran dari lapisan RNN. Eksperimen dilakukan dengan menguji berbagai kombinasi hyperparameter, termasuk ukuran hidden layer (32, 64, 128), jenis pooling, jumlah epoch (5, 50, 100), dan optimizer (SGD, RMSprop, Adam). Selain itu, variabel model_terbaik dan ringkasan_hasil disiapkan untuk menyimpan model dengan konfigurasi hyperparameter terbaik berdasarkan hasil evaluasi.

Pendekatan ini dirancang untuk mengoptimalkan performa model pada dataset tertentu. Hasil eksperimen menunjukkan bahwa peningkatan jumlah epoch memberikan perbaikan bertahap pada akurasi dan penurunan loss, baik pada data latih maupun validasi. Pada konfigurasi dengan hidden size 32, pooling maksimum, optimizer SGD, dan 5 epoch, akurasi validasi mencapai 43.42%. Ketika jumlah epoch ditingkatkan menjadi 50, akurasi validasi meningkat menjadi 46.78%, dan pada epoch 100, akurasi validasi mencapai 49.93%.

Peningkatan performa dengan bertambahnya epoch menunjukkan tren konvergensi, meskipun laju peningkatan akurasi melambat seiring waktu. Hal ini menegaskan pentingnya menentukan jumlah epoch yang optimal untuk menghindari pelatihan berlebih sekaligus memaksimalkan performa model.

